In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('additional-kiva-snapshot/loans_lenders.csv')

In [3]:
df.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


# Punto 1

In [ ]:
fin1=df['lenders'].str.split(',').explode()

In [ ]:
fin2=pd.Series(df['loan_id'])

ind={ 'loan_id': fin2 , 'Finanziatori': fin1}
fin3=pd.DataFrame(ind)

In [ ]:
fin3

# Punto 2

In [ ]:
df2= pd.read_csv('additional-kiva-snapshot/loans.csv')

In [ ]:
df2.head()

In [ ]:
df2['duration']= (df2['planned_expiration_time'].astype('datetime64') - df2['disburse_time'].astype('datetime64'))

In [ ]:
df2.duration.head()

In [ ]:
df2.planned_expiration_time.head()

In [ ]:
df2.disburse_time.head()

In [ ]:
df2.columns

In [ ]:
df2.disburse_time.isnull().value_counts

In [ ]:
df2.planned_expiration_time.isnull().value_counts

# Punto 3

In [ ]:
df3= fin3.groupby('Finanziatori').count()

In [ ]:
df3.query('loan_id > 1')

# 4


In [ ]:
df2.loc[0:10,['loan_id','loan_name','country_code','town_name']]

In [ ]:
df2.groupby('country_name')['loan_id'].count()

# 5

In [ ]:
df2.groupby('country_name')['loan_amount'].sum()

## 6

In [ ]:
 df2.groupby('country_name')['funded_amount'].sum()  /   df2.groupby('country_name')['loan_amount'].sum()

# 7

In [ ]:
df2.groupby(['country_name','disburse_time'])['funded_amount'].sum()

In [ ]:
df2.disburse_time.str.split('-', expand=True)

In [ ]:
df2['punto7'] = df2.disburse_time.str.split('-', expand= True)[0]

In [ ]:
df2.groupby(['country_name','punto7'])['funded_amount'].sum()

# 8

In [ ]:
df8= df.join(df2[['loan_idf8= dfd','loan_amount','num_lenders_total']].set_index('loan_id'), on='loan_id')
df8.head()

In [ ]:
df8['quota']= df8['loan_amount'] / df8['num_lenders_total']

In [ ]:
df8.head()

In [ ]:
df8_1= fin3.join(df8[['loan_id','quota']].set_index('loan_id'), on='loan_id')


In [ ]:
df8_1.head()

In [ ]:
df8_1.groupby('Finanziatori').quota.sum()

# Punto 9

In [ ]:
df3 = pd.read_csv('additional-kiva-snapshot/lenders.csv')

In [ ]:
df3.head()

In [ ]:
df3.groupby('country_code').size()

In [ ]:
df3.shape

In [ ]:
df3['country_code'].isnull().sum()

In [ ]:
s=df3.country_code.value_counts(normalize= True)

In [ ]:
s

In [ ]:
isnull = df3.country_code.isnull()
sample = df3.country_code.dropna().sample(isnull.sum(), replace=True).values
df3.loc[isnull, 'country_code'] = sample

In [ ]:
df3.country_code.head()

In [ ]:
df3.head()

In [ ]:
df3.columns

In [ ]:
df3.groupby('permanent_name').sum().head()

In [ ]:
df3.loc[df3.permanent_name=='muc888']

In [ ]:
df9= df8_1.groupby('Finanziatori').quota.sum()
df9.head()

In [ ]:
df9= pd.DataFrame(df9)
df9.head()

In [ ]:
df9=df9.reset_index()
df9.head()

In [ ]:
df9=df9.rename(columns={"Finanziatori": "permanent_name", "quota": "prestito_tot"})

In [ ]:
df9.head()

In [ ]:
df91= df3[['permanent_name','country_code']]

In [ ]:
df91.head()

In [ ]:
df_hope= pd.merge(df9,df91)

In [ ]:
df_hope.head()

In [ ]:
df9end = df_hope.groupby('country_code').prestito_tot.sum()

In [ ]:
df9end.head()

In [ ]:
df9end1=df2.groupby('country_code')['loan_amount'].sum()

In [ ]:
df9end = pd.DataFrame(df9end)
df9end.head()

In [ ]:
df9end1= pd.DataFrame(df9end1)
df9end1.head()


In [ ]:
df9end=df9end.reset_index()
df9end.head()

In [ ]:
df9end1 =df9end1.reset_index()
df9end1.head()

In [ ]:
df9END= df9end.join(df9end1[['country_code','loan_amount']].set_index('country_code'), on='country_code')

In [ ]:
df9END.head()

In [ ]:
df9END=df9END.fillna(0)

In [ ]:
df9END['diff'] = df9END['prestito_tot'] - df9END['loan_amount']

In [ ]:
df9END.head()

# Punto 10

In [ ]:
df10 = pd.read_csv('additional-kiva-snapshot/country_stats.csv')

In [ ]:
df10.head()

In [ ]:
df10_1 = df10[['country_code','population']]

In [ ]:
df10_1.head()

In [ ]:
df10_2=pd.merge(df10_1,df9END)

In [ ]:
df10_2.head()

In [ ]:
df10_2['punto10'] = df10_2['diff'] / df10_2['population']

In [ ]:
df10_2.head()

In [ ]:
df10_2.loc[df10_2['punto10'].idxmax(),['country_code', 'punto10']]

# Punto 11

In [ ]:
df11_1= df10[['country_code','population','population_below_poverty_line',]]

In [ ]:
df11_1.head()

In [ ]:
df11_1.population_below_poverty_line.dtype

In [ ]:
df11_1['punto11'] = df11_1['population'] * (1- df11_1['population_below_poverty_line'] / 100)

In [ ]:
df11_1.head()

In [ ]:
df11_2 = pd.merge(df11_1 , df10_2)

In [ ]:
df11_2.head()

In [ ]:
df11_2['ratio'] = df11_2['diff'] / df11_2['punto11']

In [ ]:
df11_2.loc[df11_2['ratio'].idxmax(),['country_code','ratio']]

In [ ]:
df11_2.loc[df11_2.country_code=='LU',:]

# Punto 12

In [ ]:
def day_calc(date1, date2):
    (begin,end) = (date1,date2) if date1<date2 else (date2,date1)
    date_list = list()
    date_list.append(begin)
    for y in range(begin.year+1, end.year+1):
        date_list.append(pd.Timestamp(year=y, month=1, day=1, tz='utc'))
    date_list.append(end)
    
    res = dict()
    for i in range(len(date_list)-1):
        res[date_list[i].year] = (date_list[i+1] - date_list[i]).days

    return pd.Series(res, name='DateValue')



In [ ]:
df12= pd.read_csv('additional-kiva-snapshot/loans.csv')

In [ ]:
df12['planned_expiration_time'] = pd.to_datetime(df12['planned_expiration_time'])
df12['disburse_time'] = pd.to_datetime(df12['disburse_time'])
df12['duration'] = (df12['planned_expiration_time'] - df12['disburse_time']).apply(lambda x: x.days)


In [ ]:
data_loans = df12.dropna()
df_day_weighted = data_loans.apply(lambda x: day_calc(x['planned_expiration_time'],x['disburse_time']), axis = 1)
df_day_weighted.head()

In [ ]:
df_day_weighted.fillna(0, inplace= True)
df_day_weighted = pd.concat([df_day_weighted, data_loans['loan_id']
                 , data_loans['loan_amount']
                 , data_loans['duration'] ]
                , axis = 1, sort = True)

df_day_weighted.head()

In [ ]:
final = df_day_weighted[[2011,2012,2013,2014,2015,2016,2017,2018
                        ]].div(df_day_weighted.duration, axis=0).mul(df_day_weighted.loan_amount, axis = 0)

final = pd.concat([df_day_weighted['loan_id'], final],  axis = 1 )
final = final.melt(id_vars=["loan_id"], 
        var_name="Year", 
        value_name="amount_per_year")

final.groupby(['loan_id', 'Year']).sum()

# Punto 13

In [ ]:
df13 = df2.groupby('country_code')['repayment_interval'].sum()

In [ ]:
df2.repayment_interval.head()

In [ ]:
df3.columns

In [ ]:
df2.columns

In [ ]:
def attach_totals_to_lenders():

    # {'country_code' := NaN} è considerato uno stato

    sum_by_country = df2.groupby('country_code', as_index=False).sum()[['country_code', 'funded_amount']]

    to_cross = df3

    return pd.merge(to_cross, sum_by_country, left_on='country_code', right_on='country_code')

In [ ]:
out_13 = attach_totals_to_lenders()

display(out_13)

In [ ]:
df13 = df2.groupby('country_code', as_index=False).sum()[['country_code', 'funded_amount']]

In [ ]:
pd.merge(df13,df3, left_on='country_code', right_on='country_code')

# Punto 14

In [ ]:
df3.occupation.head(50)

In [ ]:
df14_1=df3[['occupation','permanent_name']]

In [ ]:
df14_1.head()

In [ ]:
df14=df14_1.dropna()

In [ ]:
df14.head()

In [ ]:
df8_1.head()

In [ ]:
df14_3=pd.DataFrame(df8_1.groupby('Finanziatori').quota.sum())

In [ ]:
df14_3.head()

In [ ]:
df14_3=df14_3.reset_index()

In [ ]:
df14_3.head()

In [ ]:
df14 = df14.rename(columns={"occupation": "occupation", "permanent_name": "Finanziatori"})

In [ ]:
df14.head()

In [ ]:
df14_mix= df14.join(df14_3[['Finanziatori','quota']].set_index('Finanziatori'), on='Finanziatori')

In [ ]:
df14_mix.head(50)

In [ ]:
df14_mix.groupby('occupation')['quota'].sum().idxmax()


In [ ]:
df14_mix.groupby('occupation').sum()['quota'].idxmax()

# Punto 15

In [ ]:
df2[['funded_amount','disburse_time']].head(30)

In [ ]:
df2.disburse_time.str.split('-', expand=True)

In [ ]:
df2['punto7'] = df2.disburse_time.str.split('-', expand= True)[0]

In [ ]:
df2['punto15'] = df2.disburse_time.str.split('-', expand= True)[1]

In [ ]:
df2.groupby(['punto7','punto15'])['funded_amount'].sum()

# Punto 16

In [ ]:
df10.head()

In [ ]:
df10['overallGNI']= df10['population'] * df10['gni']

In [ ]:
df10.overallGNI.head(20)

# Punto 17

Find the country with the highest rate of irregular repayment interval.